In [1]:
# Configure plots for inline use in Jupyter Notebook
%matplotlib inline

import datetime as dt

# Utilities
import boto3
import dateutil
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import os

# Sentinel Hub
from sentinelhub import (
    CRS,
    BBox,
    ByocCollection,
    ByocCollectionAdditionalData,
    ByocCollectionBand,
    ByocTile,
    DataCollection,
    DownloadFailedException,
    MimeType,
    SentinelHubBYOC,
    SentinelHubRequest,
    SHConfig,
    bbox_to_dimensions,
)

config = SHConfig()
config.instance_id = os.environ.get("SH_INSTANCE_ID")
config.sh_client_id = os.environ.get("SH_CLIENT_ID")
config.sh_client_secret = os.environ.get("SH_CLIENT_SECRET")
config.aws_access_key_id = os.environ.get("username")
config.aws_secret_access_key = os.environ.get("password")
    
    
        

In [2]:
# Initialize SentinelHubBYOC class
#https://sentinelhub-py.readthedocs.io/en/latest/reference/sentinelhub.html


byoc = SentinelHubBYOC(config=config)
print ("START--------------------------")
# list collections and tiles
# from: https://sentinelhub-py.readthedocs.io/en/latest/examples/byoc_request.html
collections_iterator = byoc.iter_collections()
my_collections = list(collections_iterator)

for collection in my_collections:
    
    print("Collection name:", collection["name"])
    print("Collection id: ", collection["id"])
    tiles = list(byoc.iter_tiles(collection))
    for tile in tiles:
        print("Tile status: ", tile['status'])
        print("Tile created: ", tile['created'])
        if(tile['status'] == "FAILED"):
            print("Ingestion failed error: ", tile['additionalData'])
    print("-------------------")
print ("end")

START--------------------------
Collection name: new collection
Collection id:  9ce57124-0cc8-4c4f-9812-327ca1942170
Tile status:  INGESTED
Tile created:  2023-02-17T10:46:31.166466Z
-------------------
end


In [43]:
### Lux extend box 6.20,49.60 : 6.34,49.66

x1 = 6.20  # degree
y1 = 49.60  # degree
x2 = 6.34  # degree
y2 = 49.66  # degree

bbox = x1, y1, x2, y2
aoi = BBox([x1, y1, x2, y2], crs=CRS.WGS84)
spatial_res = 0.00018   # = 20.038 meters in degree

In [3]:
tile

{'id': '4deec260-0fc0-401d-a763-300f08414e71',
 'created': '2023-02-17T10:46:31.166466Z',
 'ingestionStart': '2023-02-17T10:46:31.166466Z',
 'coverGeometry': {'type': 'Polygon',
  'crs': {'type': 'name',
   'properties': {'name': 'urn:ogc:def:crs:EPSG::3035'}},
  'coordinates': [[[4014669.999979591, 2933829.999639364],
    [4071359.999983392, 2933829.9996399865],
    [4071359.9999840036, 3015539.999661122],
    [4014669.999980342, 3015539.999660529],
    [4014669.999979591, 2933829.999639364]]]},
 'tileGeometry': {'type': 'Polygon',
  'crs': {'type': 'name',
   'properties': {'name': 'urn:ogc:def:crs:EPSG::3035'}},
  'coordinates': [[[4014670.0, 3015540.0],
    [4071360.0, 3015540.0],
    [4071360.0, 2933830.0],
    [4014670.0, 2933830.0],
    [4014670.0, 3015540.0]]]},
 'path': 'folder/(BAND).tif',
 'status': 'INGESTED',
 'additionalData': {'hasAddedPoints': True,
  'filesMetadata': {'UrbanAtlas2018_lux': {'headerSize': 1658,
    'etag': '"517fbe1f73c3ad2b51f2939e6fc58658-2"'}},
  'mi

In [4]:
#https://sentinelhub-py.readthedocs.io/en/latest/examples/byoc_request.html
#https://eurodatacube.com/notebooks/contributions/LPIS_LULC-Slo.ipynb
data_collection = DataCollection.define_byoc(collection["id"])

In [5]:
#print(data_collection)
print(collection["name"])


new collection


In [6]:
tile_time = ["sensingTime"]

In [7]:
tile


{'id': '4deec260-0fc0-401d-a763-300f08414e71',
 'created': '2023-02-17T10:46:31.166466Z',
 'ingestionStart': '2023-02-17T10:46:31.166466Z',
 'coverGeometry': {'type': 'Polygon',
  'crs': {'type': 'name',
   'properties': {'name': 'urn:ogc:def:crs:EPSG::3035'}},
  'coordinates': [[[4014669.999979591, 2933829.999639364],
    [4071359.999983392, 2933829.9996399865],
    [4071359.9999840036, 3015539.999661122],
    [4014669.999980342, 3015539.999660529],
    [4014669.999979591, 2933829.999639364]]]},
 'tileGeometry': {'type': 'Polygon',
  'crs': {'type': 'name',
   'properties': {'name': 'urn:ogc:def:crs:EPSG::3035'}},
  'coordinates': [[[4014670.0, 3015540.0],
    [4071360.0, 3015540.0],
    [4071360.0, 2933830.0],
    [4014670.0, 2933830.0],
    [4014670.0, 3015540.0]]]},
 'path': 'folder/(BAND).tif',
 'status': 'INGESTED',
 'additionalData': {'hasAddedPoints': True,
  'filesMetadata': {'UrbanAtlas2018_lux': {'headerSize': 1658,
    'etag': '"517fbe1f73c3ad2b51f2939e6fc58658-2"'}},
  'mi

In [8]:
##https://docs.sentinel-hub.com/api/latest/evalscript/

#SentinelHubBaseApiRequest.input_data(data_collection, identifier, time_interval, maxcc, mosaicking_order, upsampling, downsampling, other_args)

evalscript_test = """

//VERSION=3
function setup() {
  return {
    input: ["B01"],
    output: { bands: 1 }
  };
}

function evaluatePixel(sample) {
  return [1 * sample.B01];
}
"""

request = SentinelHubRequest(
        evalscript=evalscript_test,
        input_data=[
            SentinelHubRequest.input_data(
                #data_collection=data_collection,
                data_collection=data_collection,
                #time_interval=tile_time
                time_interval="2018"
                
            )
        ],
        responses=[
            SentinelHubRequest.output_response('default', MimeType.PNG)
        ],
        bbox=aoi,
        size=bbox_to_dimensions(aoi, 100),
        config=config
    )

NameError: name 'aoi' is not defined

In [52]:
data = request.get_data()[0]

DownloadFailedException: Failed to download from:
https://services.sentinel-hub.com/api/v1/process
with HTTPError:
400 Client Error: Bad Request for url: https://services.sentinel-hub.com/api/v1/process
Server response: "{"status": 400, "reason": "Bad Request", "message": "CUSTOM has no band B01", "code": "RENDERER_EXCEPTION"}"